In [6]:
import ticktack
import pandas
from jax.numpy import arange, sin, pi, array, greater, less, equal, where, mean
from jax import jacrev, grad
from scipy.optimize import minimize
import matplotlib.pyplot as plt

The basic question is what is the probability of detecting consecutive events based on the distribution of the data. The first step then will be to determine the distribution of the data. This will be done be resampling the points after the event has been removed.

So I need to have a dictionary to keep track of the units for the different models.

In [8]:
model_units = { # This dictionary contains the units for the fluxes and production function
    "Guttler14": {  # Units of the Guttler 2014 paper
        "production_rate_units": "atoms/cm^2/s",    # Units of the production rate 
        "flow_rate_units": "Gt/yr"                  # Units of the fluxes
    },
    "Brehm21": {    # Units used by the Brehm, et. al. paper
        "production_rate_units": "",    # Units of the production rate 
        "flow_rate_units": ""           # Units of the fluxes
    },
    "Buntgen18": {  # The units used by the Buntgen 2018 paper
        "production_rate_units": "",    # Units of the production function
        "flow_rate_units": ""           # Units of the fluxes 
    },
    "Miyake17": {   # The units used by the Miyake 2017 et. al. paper
        "production_rate_units": "",    # Units of the production function 
        "flow_rate_units": ""           # Units of the fluxes.
    }
}

The function below will also have a `shape` parameter eventually. First however I want to get this running. Damn this really is well suited to a class structure since then I can set `self.set_annual_samples()` need to check if this has actually been implemented. The answer is __No__. I need to add the growth seasons to the `model_units` (which I might just rename `models`). This will lead to ?two? extra field `hemisphere_model` (bool) and `growth_seasons`. Alternatively this could result in a further nested dictionary like `hemispheres = {"NH_growth": array([]), "SH_growth": array([])}`

In [10]:
def get_production_function(model: str, data: str):
    params = array([1.0, 1.25, 772.0, 775.0, 40.0]) # An array containing the intial parameters of the production funtion #? This needs major work.

    cbm = ticktack.load_presaved_model( # Generating the CarbobBoxModel using ticktack
        model,  # Name of the model as looped from the models dictionary 
        production_rate_units=model_units[model]["production_rate_units"], 
        flow_rate_units=model_units[model]["flow_rate_units"]
    )
    cbm.compile()   # Generating the transfer operator via the compile() command 

    #? Need to double check that this will work
    mcmc_model = ticktack.fitting.SingleFitter(cbm)     # Fitting a model 
    mcmc_model.load_data(data)                          # Loading the data into the model 
    mcmc_model.prepare_function(model="simple_sinusoid")# Generating the simple sin model 

    sampler = mcmc_model.MarkovChainSampler(
        params, # Initial position within the parameters space    
        likelihood=mcmc_model.log_likelihood,   # likelihood function 
    )
    
    

In [11]:
get_production_function("Guttler14")

In [23]:
miyake = pandas.read_csv("Miyake12.csv", sep=" ")   # Reading the data from the Miyake 2012
DC14 = array([*miyake.d14c])                        # JAX array for manipulation
SDC14 = array([*miyake.sig_d14c])                   # JAx array for manipulation 

So below is where I minimize the $\chi^{2}$ statistic using an ODE integration style approach.

In [ ]:
sampler = mcmc_model.MarkovChainSampler(params, likelihood = mcmc_model.log_likelihood)

In [ ]:
labels = ["Amplitude ($atoms/cm^2/s$)", "Phase (yr)", "Start Date (yr)", "End Date (yr)", "Height ($atoms/cm^2/s$)"]
fig = mcmc_model.chain_summary(sampler, 20, labels=labels)

In [ ]:
mcmc_model.plot_recovery(sampler, 20)

So from here the plan is to first of all find the ideal sinusoidal model based on the data points until the event. I then run this model and subtract it away from the data allowing me to determine the signal to noise ratio of the entire data instead of a subset. Then I simulate a series of events and minimize the $\chi^{2}$ to determine the best fitting rectangular event. In addition I will have some criterion for the $\chi^{2}$ as to when an event is detected allowing me to identify the minimum parameters of the event. I will then plot a contour plot of the $\chi^{2}$ in the 2 dimensional parameter space. This will be the final product.

The next step is to minimize the $\chi^{2}$. After working so hard to get the `mcmc` I'm going to try and use this. The problem is because I was not using ticktack I can now immensly simplify the code by using the `SingleFitter` implementations provided by Q.

To minimize the $\chi^{2}$ statistic I will need to isolate the data before the event and use this to determine the $\sin$ parameters. This is going to require modifications to the `prod` function and also will require a better understanding of the plotting backend.

It will be easiest to modify the file. It is nice I must say to have got fucking nowhere all day. I'm going to go and pack now after merging these changes on github.